# Notebook Setup

In [ ]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [ ]:
# install required modules quietly
required_packages = ['azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.chdir(MY_HOME_ABS_PATH) # <------------------ ADDED
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Define paths
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = root_dir + os.sep + 'data'
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'
if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/W210/Data"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.8/387.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.3 MB/s eta 0:00:00


# Constant Definitions

In [ ]:
# Select monthly features to use
included_features= ['SITE_ID', 'year', 'month', 'TIMESTAMP',
                   'ESACCI-sm',    # ESACCI Soil Moisture (%)
                   'Percent_Snow', # Percentage of snow cover (%)
                   'NDWI',      # Normalized Different Water Index (NDWI)
                   'PET',       # Potential ET (m)
                   'MODIS_PFT', # Plant Function Type
                   'MODIS_LC',  # MODIS Land Cover
                   'MODIS_IGBP',# MODIS IGBP
                   'Ts',        # Skin temperature (K) ??
                   'LST_Day',   # Daytime land surface temperature (K)
                   'LST_Night', # Nightime land surface temperature (K)
                   'Lai',       # Leaf Area Index (LAI)
                   'Fpar',      # Fraction of photosynthetically active radiation (fPAR)
                   'CSIF-SIFdaily', # All-sky daily average SIF
                   'BESS-PAR',      # Photosynthetic Active Radiation (PAR) (W/m^2)
                   'BESS-PARdiff',  # Diffuse PAR (W/m^2)
                   'BESS-RSDN'      # Shortwave downwelling radiation (W/m^2)
                   ]

# Define in and out files for monthly data
monthly_data_input_fname = raw_data_dir + os.sep + 'data_monthly_v1_0.csv'
monthly_data_output_fname = data_dir + os.sep + "monthly-mvp.csv"

# Define methods for filling NA (interpolate or -1) and gap-fill (fill or leave be)
impute = True
impute_method = 'knn' # other options are 'interpolate', 'knn', 'constant' or None
resample_monthly = True
knn_imp_cols = ['year', 'month', 'ESACCI-sm', 'Percent_Snow', 'NDWI', 'PET', 'MODIS_LC', 'Ts', 'LST_Day',
                'LST_Night', 'Lai', 'Fpar', 'CSIF-SIFdaily', 'BESS-PAR', 'BESS-PARdiff', 'BESS-RSDN']
k=5
weights='uniform'
c=-1 # if impute_method = 'constant'

# Execute and Save Out

In [ ]:
# Execute Monthly Preparation
PrepMonthly = PrepareMonthlyData(included_features, monthly_data_input_fname, raw_data_dir+os.sep+'half_hourly_data')
monthly_df_out = PrepMonthly.run(impute, impute_method, resample_monthly, knn_imp_cols, k, weights, c)

Impute method: knn
Resampling and gap filling missing months: True
# sites dropped bc not available in data_dir: 9


33it [01:27,  1.87it/s]

CA-TP3 has column(s) with only NAN: ['ESACCI-sm']


35it [01:27,  2.49it/s]

CG-Tch has column(s) with only NAN: ['NDWI']


...

IT-Cp2 has column(s) with only NAN: ['ESACCI-sm']


232it [03:50,  1.32it/s]

DE-Hte has column(s) with only NAN: ['CSIF-SIFdaily']


234it [03:52,  1.01it/s]


Imputing values where site has 100 percent of feature missing
# of NA features before global impute: 1839
# of NA features after global impute: 0
Confirmed: No NA values remain


In [ ]:
# Save out
monthly_df_out.to_csv(monthly_data_output_fname, index=False)

In [ ]:
from google.colab import runtime
runtime.unassign()